# Utilizar TintoLib

In [1]:
import pandas as pd
import os

## Definir rutas

In [2]:
ruta_carpeta_actual = os.getcwd()
ruta_carpeta_raiz = os.path.dirname(ruta_carpeta_actual)
csv_path  = os.path.join(ruta_carpeta_raiz, "dataset", "PuntosMuestra_CR_2023.csv")
df = pd.read_csv(csv_path)

## Limpiar data

In [3]:
# Mover CATEGORIA al final
cols = [c for c in df.columns if c != "CATEGORIA"] + ["CATEGORIA"]
df = df[cols]

# Quitar columnas no predictoras (lon/lat/year) si no las quieres en la imagen
drop_cols = ["lon", "lat", "year"]
df = df[[c for c in df.columns if c not in drop_cols or c == "CATEGORIA"]]

In [4]:
df.to_csv("PuntosMuestra_CR_tinto.csv", index=False)
print(df.columns.tolist())

['B2', 'B3', 'B4', 'B8', 'B11', 'NDVI', 'EVI2', 'GCVI', 'MNDWI', 'NDMI', 'SR', 'R35', 'R54', 'CATEGORIA']


## Generar imagenes sinteticas

In [5]:
from TINTOlib.tinto import TINTO  # clase principal
import time

# df ya lo tenías preparado arriba con las bandas + 'CATEGORIA'
out_dir = "PuntosMuestra_CR_tinto_synthetic_images"
n_rows = len(df)

model = TINTO(problem="supervised", blur=True)

t0 = time.perf_counter()
model.fit_transform(df, folder=out_dir)   # <--- aquí usas df directamente
t1 = time.perf_counter()

total_s = t1 - t0
ms_per  = (total_s / n_rows) * 1000.0

print(f"Instancias procesadas: {n_rows}")
print(f"Tiempo total: {total_s:.2f} s ({total_s/60:.2f} min)")
print(f"Tiempo promedio por instancia: {ms_per:.1f} ms/instancia")

Instancias procesadas: 29201
Tiempo total: 37.38 s (0.62 min)
Tiempo promedio por instancia: 1.3 ms/instancia


# Registrar informacion

In [6]:
import sys
sys.path.append("..")
import importlib, utils_log
importlib.reload(utils_log)
from utils_log import log_row

carpeta_actual = ruta_carpeta_actual.split("\\")[-1]
dataset_utilizado = csv_path.split("\\")[-1]

log_row(
  script="20250901_ConversionTINTO.ipynb",
  algoritmo="TINTO_PCA",
  dataset="PuntosMuestra_CR_tinto.csv",
  clases_removidas=[], seed=1,
  n_train=None, n_test=None,
  n_features=df.shape[1]-1, num_classes=df["CATEGORIA"].nunique(),
  params={"pixels":30,"algorithm":"PCA","blur":True},
  fit_seconds=total_s, pred_seconds=0,
  ms_per_sample=(total_s/len(df))*1000,
  carpeta=carpeta_actual
)

C:\Users\Francessca\anaconda3\envs\tintoenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
